<a href="https://colab.research.google.com/github/aarushiwagh/Domain_Specific_Financial_LLM/blob/master/Falcon_7b_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup

In [ ]:
#!pip install -Uqqq pip
#!pip install -qqq bitsandbytes==0.39.0
#!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install transformers

In [ ]:
!pip install bitsandbytes

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [ ]:
prompt = """
<human>: I need to perform financial due diligence for a potential acquisition. Can you provide me with a checklist and recommend any relevant tools?
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: I need to perform financial due diligence for a potential acquisition. Can you provide me with a checklist and recommend any relevant tools?
<assistant>: Sure, I can help you with that. Here's a checklist for financial due diligence:
- Review the company's financial statements and tax returns
- Analyze the company's cash flow and revenue streams
- Perform a valuation of the company
- Review the company's legal and tax compliance
- Assess the company's competitive landscape
- Analyze the company's intellectual property and contracts
- Perform a risk assessment
- Review the company's financial projections

As for tools, there are several financial due diligence software options available such as DueDiligence, Kroll, and Valuation Partners. These tools can help you streamline the process and ensure accuracy.
User 
CPU times: user 15.3 s, sys: 0 ns, total: 15.3 s
Wall time: 15.3 s


# Prep dataset

In [ ]:
from datasets import Dataset
Dataset.cleanup_cache_files

<function datasets.arrow_dataset.Dataset.cleanup_cache_files(self) -> int>

In [ ]:
data = load_dataset("csv", data_files="/content/fin_data_fin1.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 998
    })
})

In [ ]:
data["train"][0]

{'instruction': 'For a car, what scams can be plotted with 0% financing vs rebate?',
 'output': "The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 36,48,60 or even 72 months. The shorter length ma

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["instruction"]}
<assistant>: {data_point["output"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['instruction', 'output', 'input_ids', 'attention_mask'],
    num_rows: 998
})

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.026400
2,2.986300
3,2.703100
4,2.916200
5,2.825700
6,2.843500
7,2.856600
8,2.757600
9,2.982700
10,2.925300


Step,Training Loss
1,3.026400
2,2.986300
3,2.703100
4,2.916200
5,2.825700
6,2.843500
7,2.856600
8,2.757600
9,2.982700
10,2.925300


TrainOutput(global_step=249, training_loss=2.5840892973673872, metrics={'train_runtime': 2104.4853, 'train_samples_per_second': 0.474, 'train_steps_per_second': 0.118, 'total_flos': 5139798899649024.0, 'train_loss': 2.5840892973673872, 'epoch': 1.0})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "aarushiwagh/financial-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aarushiwagh/financial-falcon-7b/commit/3e08eeab9fe36fe332bdedcfa3acfd802fafb5e0', commit_message='Upload model', commit_description='', oid='3e08eeab9fe36fe332bdedcfa3acfd802fafb5e0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: Analyze the sentiment of the following financial sentence and answer only as 'positive', 'negative' or 'neutral': For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Analyze the sentiment of the following financial sentence and answer only as 'positive', 'negative' or 'neutral': For the last quarter of 2010, Componenta's net sales doubled to EUR131m from EUR76m for the same period a year earlier, while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m.
<assistant>: Positive.  The company's sales doubled, and they moved from a loss to a zero pre-tax profit.  That's a pretty good result.  The sentence is positive.  The sentence is also neutral, since it doesn't mention any other factors that might have affected the company's performance.  The sentence is also positive, since it doesn't mention any other factors that might have affected the company's performance.  The sentence is also neutral, since it doesn't mention any other factors that might have affected the company's performance.  The sentence is also positive, since it doesn't mention any other factors that might have affected the company's performance.  The sentence is a

## Evaluate the model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

data = pd.read_csv('/content/data.csv')
data = data.head(100)

In [ ]:
X = data["Sentence"]
y = data["Sentiment"]

# Function to evaluate LM accuracy
def evaluate_lm_accuracy(lm, X_data, y_true):
    y_pred = []
    for prompt in X_data:
      prompt = f"Analyze the sentiment of the following sentence and answer in one word as 'positive', 'negative' or 'neutral': {prompt}"
      #print(prompt)
      ############inference code###########################
      encoding = tokenizer(prompt, return_tensors="pt").to(device)
      with torch.inference_mode():
        outputs = lm.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )
      #print("1 eval done")
      #print("rfind: ",tokenizer.decode(outputs[0], skip_special_tokens=True))
      #print(tokenizer.decode(outputs[0], skip_special_tokens=True))
      parts = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n')
      #print(parts[1])
      y_pred.append(parts[1])
      #print(y_pred)
    ####################################################################
    accuracy = accuracy_score(y_true, y_pred)
    #precision = precision_score(y_true,y_pred)
    #recall = recall_score(y_true,y_pred)
    #f1_score = f1_score(y_true,y_pred)
    return accuracy

metrics_lm1 = evaluate_lm_accuracy(model, X, y)
#metrics_lm2 = evaluate_lm_accuracy(lm_model_2, X, y)

print(f"Metrics of model 1: {metrics_lm1}")
#print(f"Metrics of model 2: {metrics_lm2}")


Metrics of model 1: 0.19


In [ ]:
X = data["Sentence"]
y = data["Sentiment"]

def detect_sentiment(text):
    sentiment_words = ['positive', 'negative', 'neutral']

    # Convert the text to lowercase for case-insensitive matching
    lowercase_text = text.lower()

    # Check if any of the sentiment words is present in the text
    for sentiment_word in sentiment_words:
        if sentiment_word in lowercase_text:
            return sentiment_word

    # If none of the sentiment words is found, return None or any default value you prefer
    return None

# Function to evaluate LM accuracy
def evaluate_lm_accuracy(lm, X_data, y_true):
    y_pred = []
    for prompt in X_data:
      prompt = f"Analyze the sentiment of the following financial sentence and answer only as 'positive', 'negative' or 'neutral': {prompt}"
      #print(prompt)
      ############inference code###########################
      encoding = tokenizer(prompt, return_tensors="pt").to(device)
      with torch.inference_mode():
        outputs = lm.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )
      #print("1 eval done")
      #print("rfind: ",tokenizer.decode(outputs[0], skip_special_tokens=True))
      #print(tokenizer.decode(outputs[0], skip_special_tokens=True))
      #parts = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n')
      #print(parts[1])
      op = tokenizer.decode(outputs[0], skip_special_tokens=True)
      op_word = detect_sentiment(op)
      y_pred.append(op_word)
      #print(y_pred)
    ####################################################################
    accuracy = accuracy_score(y_true, y_pred)
    #precision = precision_score(y_true,y_pred)
    #recall = recall_score(y_true,y_pred)
    #f1_score = f1_score(y_true,y_pred)
    return accuracy

metrics_lm1 = evaluate_lm_accuracy(model, X, y)
#metrics_lm2 = evaluate_lm_accuracy(lm_model_2, X, y)

print(f"Metrics of model 1: {metrics_lm1}")
#print(f"Metrics of model 2: {metrics_lm2}")


Metrics of model 1: 0.33


In [ ]:
import csv

path = '/content/fin_data2.csv'
output_csv_path = '/content/decoded_output.csv'

# Open the input file in binary mode for reading
with open(path, 'rb') as reader:

    # Open the output CSV file for writing
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Iterate over each byte in the input file
        for utf8_bytes in reader:

            # Decode the byte sequence as UTF-8, ignoring decoding errors
            line = utf8_bytes.decode('utf-8', 'ignore')

            # Write the decoded line to the CSV file
            csv_writer.writerow([line])

# Print a message indicating the CSV file has been created
print(f'Decoded lines saved to: {output_csv_path}')

Decoded lines saved to: /content/decoded_output.csv


In [ ]:
import pandas as pd
df = pd.read_csv('fin_data2.csv')
df['output'] = df['output'].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))

In [ ]:
df.to_csv("1.csv")